### 1. Predictive Performance Monitoring

#### 1.1 Batch Inference Function

In [ ]:
def run_inference(model, X, y_true=None):
    y_pred = model.predict_proba(X)[:, 1]
    if y_true is not None:
        auc = roc_auc_score(y_true, y_pred)
        print(f"AUC: {auc:.4f}")
    return y_pred

#### 1.2 Drift and Performance Report

In [ ]:
def monitor_performance(model, X_new, y_new, threshold_auc=0.7):
    y_pred = run_inference(model, X_new, y_new)
    auc = roc_auc_score(y_new, y_pred)
    if auc < threshold_auc:
        print("Model performance degraded. Consider recalibration or retraining.")
    else:
        print("Model performance acceptable.")

### 2. Recalibration Monitoring

#### 2.1 Recalibration

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

def calibrate_model(model, X_val, y_val):
    calibrator = CalibratedClassifierCV(model, cv='prefit')
    calibrator.fit(X_val, y_val)
    return calibrator

#### 2.2 Comparison

In [ ]:
def plot_calibration(y_true, y_pred):
    sns.histplot(y_pred, bins=20, kde=True, label='Predicted Probabilities')
    sns.histplot(y_true, bins=2, kde=False, label='True Labels', color='orange')
    plt.legend()
    plt.title("Calibration Check")
    plt.show()

### 3. Schema Drift

In [ ]:
EXPECTED_COLUMNS = ['col1', 'col2', 'col3', 'col4']

def check_new_columns(df):
    new_cols = set(df.columns) - set(EXPECTED_COLUMNS)
    missing_cols = set(EXPECTED_COLUMNS) - set(df.columns)

    if new_cols:
        print(f"New columns detected: {new_cols}")
    if missing_cols:
        print(f"Missing expected columns: {missing_cols}")
    if not new_cols and not missing_cols:
        print("Schema matches expected input.")

#### 4. Maintainance Pipeline Execution

In [ ]:
def full_model_maintenance(model, new_data_df, y_true=None):
    check_new_columns(new_data_df)
    monitor_performance(model, new_data_df[EXPECTED_COLUMNS], y_true)